In [ ]:
#Necessary Dependencies:
#!pip install gym_tetris
#!pip install nes-py

In [ ]:
#Necessary Dependencies:
#pip install gym_tetris
from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT
import numpy as np
import matplotlib.pyplot as plt
import time

#Necessary constant
#This implementation uses the NTSC version of tetris, which has slightly different frames for the falling pieces than the PAL version.
#Source for fall frames: https://listfist.com/list-of-tetris-levels-by-speed-nes-ntsc-vs-pal
SpeedtoFallFrames = [48, 43, 38, 28, 23, 18, 13, 8, 6, 5, 5, 5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]

#This is what corresponds to the piece location/rotation, which is found via self.env.ram[0x0042].
#It's not particularly useful for our purposes because info["current_piece"] is just outright better, but it is still noteworthy.
PieceOrientation = {
    "T" : [0, 1, 2, 3],
    "J" : [4, 5, 6, 7],
    "Z" : [8, 9],
    "O" : [10],
    "S" : [11, 12],
    "L" : [13, 14, 15, 16],
    "I" : [17, 18]
    
}



#Takes the board ripped out from the NES RAM and converts it to a simpler board to use
#Is mostly necessary because the board in the NES RAM also takes into account the colors of the pieces, which is not necessary for the algorithm to know.
def ColorBoardtoSimpleBoard(board):
    simpleboard = []
    for row in board:
        simplerow = []
        for cell in row:
            if cell == 239:
                simplerow.append(0)
            else:
                simplerow.append(1)
        simpleboard.append(tuple(simplerow))
    return tuple(simpleboard)

def CY(var):
    return min(19,max(var,0))
def CX(var):
    return min(9,max(var,0))
#Used to check collision for pieces
               
#True = No problem, False = piece is inside blocks
def CollisionDetection(boardstate, currentpiece, piecelocation):
    x = int(piecelocation[0])
    y = int(piecelocation[1])
    #If x/y are outside of acceptable boundaries, directly return False
    if x != CX(x) or y != CY(y):
        return False
    
    
    if currentpiece == "O":
        if boardstate[y][x] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y)][CX(x-1)] == 0 and boardstate[CY(y+1)][CX(x-1)] == 0:
            return True
        else:
            print(boardstate[y][x], boardstate[CY(y+1)][CX(x)], boardstate[CY(y)][CX(x-1)], boardstate[CY(y+1)][CX(x-1)])
            return False
    elif currentpiece == "Ju":
        if boardstate[y][x] == 0 and boardstate[CY(y)][CX(x+1)] == 0 and boardstate[CY(y)][CX(x-1)] == 0 and boardstate[CY(y-1)][CX(x-1)] == 0:
            return True
        else:
            print(boardstate[y][x], boardstate[CY(y)][CX(x+1)], boardstate[CY(y)][CX(x-1)], boardstate[CY(y-1)][CX(x-1)])
            return False
    elif currentpiece == "Jd":
        if boardstate[y][x] == 0 and boardstate[CY(y)][CX(x-1)] == 0 and boardstate[CY(y)][CX(x+1)] == 0 and boardstate[CY(y+1)][CX(x+1)] == 0:
            return True
        else:
            print(boardstate[y][x], boardstate[CY(y)][CX(x-1)], boardstate[CY(y)][CX(x+1)], boardstate[CY(y+1)][CX(x+1)])
            return False
    elif currentpiece == "Jl":
        if boardstate[y][x] == 0 and boardstate[CY(y-1)][CX(x)] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y+1)][CX(x-1)] == 0:
            return True
        else:
            print(boardstate[y][x], boardstate[CY(y-1)][CX(x)], boardstate[CY(y+1)][CX(x)], boardstate[CY(y+1)][CX(x-1)])
            return False
    elif currentpiece == "Jr":
        if boardstate[y][x] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y-1)][CX(x)] == 0 and boardstate[CY(y-1)][CX(x+1)] == 0:
            return True
        else:
            print(boardstate[y][x], boardstate[CY(y+1)][CX(x)], boardstate[CY(y-1)][CX(x)], boardstate[CY(y-1)][CX(x+1)])
            return False
    elif currentpiece == "Zh":
        if boardstate[y][x] == 0 and boardstate[CY(y)][CX(x-1)] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y+1)][CX(x+1)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Zv":
        if boardstate[y][x] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y)][CX(x+1)] == 0 and boardstate[CY(y-1)][CX(x+1)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Sh":
        if boardstate[y][x] == 0 and boardstate[CY(y)][CX(x+1)] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y+1)][CX(x+1)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Sv":
        if boardstate[y][x] == 0 and boardstate[CY(y-1)][CX(x)] == 0 and boardstate[CY(y)][CX(x+1)] == 0 and boardstate[CY(y+1)][CX(x+1)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Ih":
        if boardstate[y][x] == 0 and boardstate[CY(y)][CX(x+1)] == 0 and boardstate[CY(y)][CX(x-1)] == 0 and boardstate[CY(y)][CX(x-2)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Iv":
        if boardstate[y][x] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y-1)][CX(x)] == 0 and boardstate[CY(y-2)][CX(x)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Td":
        if boardstate[y][x] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y)][CX(x-1)] == 0 and boardstate[CY(y)][CX(x+1)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Tl":
        if boardstate[y][x] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y)][CX(x-1)] == 0 and boardstate[CY(y-1)][CX(x)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Tr":
        if boardstate[y][x] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y)][CX(x+1)] == 0 and boardstate[CY(y-1)][CX(x)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Tu":
        if boardstate[y][x] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y)][CX(x+1)] == 0 and boardstate[CY(y)][CX(x-1)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Ld":
        if boardstate[y][x] == 0 and boardstate[CY(y)][CX(x+1)] == 0 and boardstate[CY(y)][CX(x-1)] == 0 and boardstate[CY(y-1)][CX(x-1)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Ll":
        if boardstate[y][x] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y-1)][CX(x)] == 0 and boardstate[CY(y-1)][CX(x-1)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Lr":
        if boardstate[y][x] == 0 and boardstate[CY(y-1)][CX(x)] == 0 and boardstate[CY(y+1)][CX(x)] == 0 and boardstate[CY(y+1)][CX(x+1)] == 0:
            return True
        else:
            return False
    elif currentpiece == "Lu":
        if boardstate[y][x] == 0 and boardstate[CY(y)][CX(x-1)] == 0 and boardstate[CY(y)][CX(x+1)] == 0 and boardstate[CY(y-1)][CX(x+1)] == 0:
            return True
        else:
            return False
    else:
        #Not an accepted piecetype
        return False

#To use this class, simply run the following line in Agent's action function:
#MicroState(ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()), self.info, tuple(self.env.ram[0x0040:0x0042]), self.env.ram[0x0044], "NOOP")
#A* Algorithm can then be run via running takeAction function 
class MicroState:
    
    def __init__(self, boardstate, currentpiece, piecelocation, speed, previousaction):
        self.boardstate = boardstate
        self.currentpiece = currentpiece
        self.piecelocation = piecelocation
        self.speed = speed
        self.fallframes = SpeedtoFallFrames[min(speed, 29)] - 1
        self.softdrop = 0
        self.previousaction = previousaction
    
    def takeAction(self, action):
        bs = self.boardstate
        cp = self.currentpiece
        pl = self.piecelocation
        ff = self.fallframes
        sd = self.softdrop
        if "down" in action:
            if sd == 1 and ff > 1:
                pl[1] += 1
                sd = 0
            elif sd == 0:
                sd = 1
        else:
            sd = 0
        if "left" in action and "left" not in previousaction:
            pl[0] -= 1
            if not CollisionDetection(bs, self.currentpiece, pl):
                pl[0] += 1
        if "right" in action and "right" not in previousaction:
            pl[0] += 1
            if not CollisionDetection(bs, self.currentpiece, pl):
                pl -= 1
        if "A" in action and "A" not in previousaction:
            if cp[0] != "O":
                if cp[1] == "h":
                    cp[1] = "v"
                elif cp[1] == "v":
                    cp[1] = "h"
                elif cp[1] == "u":
                    cp[1] = "r"
                elif cp[1] == "r":
                    cp[1] = "d"
                elif cp[1] == "d":
                    cp[1] = "l"
                elif cp[1] == "l":
                    cp[1] = "u"
        
        if "B" in action and "B" not in previousaction:
            if cp[0] != "O":
                if cp[1] == "h":
                    cp[1] = "v"
                elif cp[1] == "v":
                    cp[1] = "h"
                elif cp[1] == "u":
                    cp[1] = "l"
                elif cp[1] == "l":
                    cp[1] = "d"
                elif cp[1] == "d":
                    cp[1] = "r"
                elif cp[1] == "r":
                    cp[1] = "u"
        
        ff -= 1
        if ff <= 0:
            pl[1] += 1
        m = MicroState(bs, cp, pl, self.speed, action)
        m.fallframes = ff
        m.softdrop = sd
        return m
            

        
#To use this class, simply run the following line in Agent's action function:
##MacroState(ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()), self.info, self.nextinfo)
class MacroState:
    
    def __init__(self, boardstate, currentpiece, nextpiece):
        self.boardstate = boardstate
        self.currentpiece = currentpiece
        self.nextpiece = nextpiece

class Agent:
    
    def __init__(self, episodes=1):
        self.env = gym_tetris.make('TetrisA-v2',deterministic = True)
        self.env = JoypadSpace(self.env, MOVEMENT)
        #self.env.deterministic = True
        #Testing to see whether using the pixels for the state works better than just the board.
        #self.state = self.env.reset()
        self.env.reset()
        #self.env.deterministic = True
        self.env.render()
        self.state = tuple([ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()), "", "", tuple(self.env.ram[0x0040:0x0042])])
        #print(self.env.ram[0x0044])
        self.info = "NONE"
        self.nextinfo = "NONE"
        self.prevaction = "NOOP"
        self.highscore = 0
        self.time = 0
        self.linestates = []
        self.listofhighscores = []
        self.listofhighscorerates = []
        self.listofsafetyscores = []
        
        self.actions = MOVEMENT
        self.state_actions = []  # state & action track

        self.episodes = episodes  # number of episodes going to play
        self.steps_per_episode = []
        
        
        
    def chooseAction(self):
        action = 0
        
        #print(self.actions)
        if self.info == "NONE":
            action = np.random.choice(len(self.actions))
        else:
            #Algorithms go here
            j = "j"
            #MicroState(ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()), self.info, tuple(self.env.ram[0x0040:0x0042]), self.env.ram[0x0044], self.prevaction)
            #MacroState(ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()), self.info, self.nextinfo)
        self.prevaction = action
        return action
    
    def reset(self):
        self.env.close()
        self.env = gym_tetris.make('TetrisA-v2',deterministic = True)
        self.env = JoypadSpace(self.env, MOVEMENT)
        #self.env.deterministic = True
        self.env.reset()
        #self.env.deterministic = True
        self.env.render()
        self.state = tuple([ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()), "", "", tuple(self.env.ram[0x0040:0x0042])])
        self.info = "NONE"
        self.nextinfo = "NONE"
        self.prevaction = "NOOP"
        #Conversion of self.state to tuple for hashing purposes
        #self.state = tuple([tuple(x) for x in self.state])
        self.state_actions = []
        self.highscore = 0
        self.time = 0
        self.linestates = []


    def play(self):
        self.steps_per_episode = []  
        
        for ep in range(self.episodes):
            done = False
            while not done:

                action = self.chooseAction()
                self.state_actions.append((self.state, action))
                

                #unusedstate, reward, done, info = self.env.step(self.env.action_space.sample())
                
                unusedstate, reward, done, info = self.env.step(action)
                
                #print(self.env.ram[0x0045])
                #try:
                #    if previnfo[0] != info["current_piece"][0]:
                #        print(previnfo)
                #        print(prevpos)
                #except:
                #    j = "j"
                #print(info["current_piece"])
                #print(info["next_piece"])
                #print(self.env.ram[0x0042])
                
                
                self.env.render()
                self.state = tuple([ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()), info["current_piece"], info["next_piece"], tuple(self.env.ram[0x0040:0x0042])])
                self.info = info["current_piece"]
                self.nextinfo = info["next_piece"]
                #if CollisionDetection(self.state[0], self.state[1], self.state[3]) == False:
                    #print(self.state[1])
                    #print(self.state[3])
                    #time.sleep(10)
                
                self.highscore = info["score"]
                self.time += 1
                self.linestates.append(info["board_height"])

            # end of game
            #if ep % 10 == 0:
            self.listofhighscores.append(self.highscore)
            self.listofhighscorerates.append(self.highscore / self.time)
            self.listofsafetyscores.append(sum(self.linestates) / self.time)
            print("episode", ep)
            print("Highscore: " + str(self.highscore))
            print("Score rate: " + str(self.highscore / self.time))
            print("Safety score: " + str(sum(self.linestates) / self.time))
            self.steps_per_episode.append(len(self.state_actions))
            #print(ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()))
            self.reset()
        self.env.close()
if __name__ == "__main__":
    N_EPISODES = 1
    # comparison
    agent = Agent(episodes=N_EPISODES)
    agent.play()

    highscores = agent.listofhighscores
    highscorerates = agent.listofhighscorerates
    safetyscores = agent.listofsafetyscores

    plt.figure(figsize=[10, 6])
    plt.ylim(0, 50)
    plt.plot(range(N_EPISODES), highscores, label="high score")
    plt.legend()
        
    plt.figure(figsize=[10, 6])
    plt.ylim(0, 0.1)
    plt.plot(range(N_EPISODES), highscorerates, label="score rate")
    plt.legend()
    
    plt.figure(figsize=[10, 6])
    plt.ylim(4, 12)
    plt.plot(range(N_EPISODES), safetyscores, label="safety score")
    plt.legend()

